<a href="https://colab.research.google.com/github/mehwishferoz/EIS/blob/master/CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing necessary libraries



In [ ]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F
from tqdm.auto import tqdm
import numpy as np
import os
try:
    import torchinfo
except:
    !pip install torchinfo
    import torchinfo

from torchinfo import summary

### Setting values for retraining


In [ ]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(42)

### Setting the component for the entire process
GPU if available, otherwise CPU

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Loading data

In [ ]:
BATCH_SIZE = 64
NUM_WORKERS = os.cpu_count()

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

100%|██████████| 169001437/169001437 [00:13<00:00, 12767510.12it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


### Defining methods for training and testing

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [ ]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            test_pred_logits = model(X)

            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [ ]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):

    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

### Defining constants


In [ ]:
EPOCHS = 200

### Creating the main model class

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class Net(nn.Module):
  def __init__(self, in_channels, num_classes):
    super(Net, self).__init__()
    self.conv1 = conv_block(in_channels, 64)
    self.conv2 = conv_block(64, 128, pool=True)
    self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

    self.conv3 = conv_block(128, 256, pool=True)
    self.conv4 = conv_block(256, 512, pool=True)
    self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
    self.conv5 = conv_block(512, 1028, pool=True)
    self.res3 = nn.Sequential(conv_block(1028, 1028), conv_block(1028, 1028))

    self.classifier = nn.Sequential(nn.MaxPool2d(2), # 1028 x 1 x 1
                                    nn.Flatten(), # 1028
                                    nn.Linear(1028, num_classes)) # 1028 -> 100

  def forward(self, xb):
    out = self.conv1(xb)
    out = self.conv2(out)
    out = self.res1(out) + out
    out = self.conv3(out)
    out = self.conv4(out)
    out = self.res2(out) + out
    out = self.conv5(out)
    out = self.res3(out) + out
    out = self.classifier(out)
    return out

### Instantiating the model

In [ ]:
model = Net(3, 100).to(device)

### Model overview

In [ ]:
summary(model)

Layer (type:depth-idx)                   Param #
Net                                      --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       1,792
│    └─BatchNorm2d: 2-2                  128
│    └─ReLU: 2-3                         --
├─Sequential: 1-2                        --
│    └─Conv2d: 2-4                       73,856
│    └─BatchNorm2d: 2-5                  256
│    └─ReLU: 2-6                         --
│    └─MaxPool2d: 2-7                    --
├─Sequential: 1-3                        --
│    └─Sequential: 2-8                   --
│    │    └─Conv2d: 3-1                  147,584
│    │    └─BatchNorm2d: 3-2             256
│    │    └─ReLU: 3-3                    --
│    └─Sequential: 2-9                   --
│    │    └─Conv2d: 3-4                  147,584
│    │    └─BatchNorm2d: 3-5             256
│    │    └─ReLU: 3-6                    --
├─Sequential: 1-4                        --
│    └─Conv2d: 2-10                      295,168
│

### Training the model and freezing layers

In [ ]:
train(model=model,
          train_dataloader=train_loader,
          test_dataloader=test_loader,
          optimizer=torch.optim.Adam(params=model.parameters(), lr=0.0001),
          loss_fn=F.cross_entropy,
          epochs=EPOCHS)

  0%|          | 0/200 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 1 | train_loss: 3.3414 | train_acc: 0.1997 | test_loss: 2.7383 | test_acc: 0.3168
Epoch: 2 | train_loss: 2.4524 | train_acc: 0.3627 | test_loss: 2.2298 | test_acc: 0.4164
Epoch: 3 | train_loss: 2.0375 | train_acc: 0.4538 | test_loss: 1.9077 | test_acc: 0.4858
Epoch: 4 | train_loss: 1.7741 | train_acc: 0.5146 | test_loss: 1.8625 | test_acc: 0.4946
Epoch: 5 | train_loss: 1.5821 | train_acc: 0.5598 | test_loss: 1.7040 | test_acc: 0.5338
Epoch: 6 | train_loss: 1.4273 | train_acc: 0.5969 | test_loss: 1.5783 | test_acc: 0.5669
Epoch: 7 | train_loss: 1.2996 | train_acc: 0.6291 | test_loss: 1.5580 | test_acc: 0.5753
Epoch: 8 | train_loss: 1.1843 | train_acc: 0.6554 | test_loss: 1.4789 | test_acc: 0.5944
Epoch: 9 | train_loss: 1.0829 | train_acc: 0.6857 | test_loss: 1.4575 | test_acc: 0.6006
Epoch: 10 | train_loss: 0.9872 | train_acc: 0.7104 | test_loss: 1.3784 | test_acc: 0.6217
Epoch: 11 | train_loss: 0.9000 | train_acc: 0.7337 | test_loss: 1.3901 | test_acc: 0.6166
Epoch: 12 | train_l

KeyboardInterrupt: 

### Saving the model

In [ ]:
torch.save(model.state_dict(), 'models/model.pt')

### Loading the model

In [ ]:
model_normal = Net(3, 100).to(device)
model_normal.load_state_dict(torch.load('models/model.pt'))

<All keys matched successfully>

In [ ]:
import torch
from torch.utils.data import DataLoader

# Load the model
model = torch.load('models/model.pt')

In [ ]:
import torch
from torch.utils.data import DataLoader

def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_normal = Net(3, 100).to(device)
model_normal.load_state_dict(torch.load('models/model.pt'))
model_normal.eval()

test_accuracy = evaluate(model_normal, test_loader, device
print('Test Accuracy: {:.2f}%'.format(test_accuracy * 100))


Test Accuracy: 68.96%
